In [1]:
# Define a color scheme for different types
color_map = {
    '"ORGANIZATION"': 'blue',
    '"EVENT"': 'green',
    '"PERSON"': 'purple',
    '"GEO"': 'red',
    # Add more types and colors as needed
}

In [2]:
import networkx as nx
import json
import ipywidgets as widgets
from IPython.display import display


# Load the .graphml file
graphml_file = '/home/ljc/graphrag/ragtest/output/20240801-183929/artifacts/merged_graph.graphml'  # Replace with the actual path to your .graphml file
graph = nx.read_graphml(graphml_file)

# Define a color scheme for different types
color_map = {
    'ORGANIZATION': '#008B8B',
    'EVENT': '#8FBC8F',
    'PERSON': '#E9967A',
    'GEO': '#A9A9A9',
    # Add more types and colors as needed
}

# Extract nodes and edges, cleaning up the escape characters
nodes = [{'id': str(node_id).strip('"'), 'type': data.get('type', '').strip('"')} for node_id, data in graph.nodes(data=True)]
edges = [{'source': str(edge[0]).strip('"'), 'target': str(edge[1]).strip('"')} for edge in graph.edges()]

# Convert to JSON format required by 3d-force-graph
graph_data = {
    'nodes': nodes,
    'links': edges
}

# Convert the graph data to JSON string
graph_data_json = json.dumps(graph_data, ensure_ascii=False)

# Create the HTML content for 3d-force-graph
html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <style> body {{ margin: 0; }} </style>
    <script src="https://unpkg.com/3d-force-graph"></script>
</head>
<body>
    <div id="3d-graph" style="width: 100vw; height: 100vh;"></div>

    <script type="importmap"> 
    {{
        "imports": {{ 
            "three": "https://unpkg.com/three/build/three.module.js" 
        }}
    }}
    </script>
    <script type="module">
        import SpriteText from "https://unpkg.com/three-spritetext/dist/three-spritetext.mjs";
        import * as d3 from "https://unpkg.com/d3@6?module";

        const graphData = {graph_data_json};

        const colorMap = {{
            'ORGANIZATION': '#008B8B',
            'EVENT': '#8FBC8F',
            'PERSON': '#E9967A',
            'GEO': '#A9A9A9',
            '': 'gray'  // Default color for undefined types
        }};

        const Graph = ForceGraph3D()
            (document.getElementById('3d-graph'))
                .graphData(graphData)
                .nodeAutoColorBy('type')
                .nodeThreeObject(node => {{
                    const sprite = new SpriteText(node.id);
                    sprite.material.depthWrite = false; // make sprite background transparent
                    sprite.color = colorMap[node.type] || 'gray';
                    sprite.textHeight = 8;
                    return sprite;
                }});

        // Increase repulsion force and link distance
        Graph.d3Force('charge', d3.forceManyBody().strength(-300));
        Graph.d3Force('link', d3.forceLink().distance(200));


    </script>
</body>
</html>
"""

# Display the HTML content using ipywidgets
html_widget = widgets.HTML(value=html_content)



In [4]:
graph_data_json

'{"nodes": [{"id": "OPENAI", "type": "ORGANIZATION"}, {"id": "SAM ALTMAN", "type": "PERSON"}, {"id": "LAWRENCE SUMMERS", "type": "PERSON"}, {"id": "PAUL NAKASONE", "type": "PERSON"}, {"id": "SAN FRANCISCO", "type": "GEO"}, {"id": "MICROSOFT", "type": "ORGANIZATION"}, {"id": "CHATGPT", "type": "EVENT"}, {"id": "OPENAI FIVE", "type": "EVENT"}, {"id": "DALL-E", "type": "EVENT"}, {"id": "SORA", "type": "EVENT"}, {"id": "OPENAI GLOBAL, LLC", "type": "ORGANIZATION"}, {"id": "BRET TAYLOR", "type": "PERSON"}, {"id": "ARTIFICIAL INTELLIGENCE", "type": "CONCEPT"}, {"id": "AI BOOM", "type": "EVENT"}, {"id": "LAWSUITS", "type": "EVENT"}, {"id": "OPENAI, INC.", "type": ""}, {"id": "TIM COOK", "type": "PERSON"}, {"id": "APPLE", "type": "ORGANIZATION"}, {"id": "2024", "type": "EVENT"}, {"id": "INNOVATIVE VISION", "type": "CONCEPT"}, {"id": "LEADERSHIP CHANGE", "type": ""}], "links": [{"source": "OPENAI", "target": "SAM ALTMAN"}, {"source": "OPENAI", "target": "MICROSOFT"}, {"source": "OPENAI", "targe

In [3]:
# Save the HTML content to a file
with open('text_node.html', 'w') as f:
    f.write(html_content)
